<html>
    <summary></summary>
         <div> <p></p> </div>
         <div style="font-size: 20px; width: 800px;"> 
              <h1>
               <left>Intro to Bioinformatics in Python: Downloading and Annotating Genomic Sequences.</left>
              </h1>
              <p><left>============================================================================</left> </p>
<pre>Course: BIOM/CBE 480A5, Spring 2025
Instructor: Brian Munsky
Contact Info: munsky@colostate.edu
</pre>
         </div>
    </p>

</html>

<details>
  <summary>Copyright info</summary>

```
Copyright 2024 Brian Munsky

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```
<details>



In [15]:
# %pip install biopython
# %pip install Bio
import Bio
from Bio import Entrez, SeqIO, pairwise2, AlignIO, Phylo
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import ProtParam
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import webbrowser
#
import os
import threading

from Bio import Align
from Bio.Align import Alignment
from pymsaviz import MsaViz

In [16]:
#(PRELIMINARIES) Let's download the COVID19 virus sequence from NCBI
# Provide your email for accessing NCBI
Entrez.email = "hayley.stern@colostate.edu"  # Insert your email here

def get_genbank(accession_number):
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

corona_virus = get_genbank("MN908947")

In [17]:
# Let's get our protein of interest from last time.

# Function to find ORFs in all frames of a sequence
def find_all_orfs(sequence):
    orf_list = []           # List of all the ORF we find
    for frame in range(3):  # Check each of the three reading frames
        orfs = []           # List of ORFs for the current frame   
        lastStop = frame    # Position of last stop codon.
        for orf in sequence[frame:].translate(to_stop=False).split('*'):     # Translate the curent frame sequence into AA and break up by stop codons
            if len(orf) > 30:                                                # Consider ORFs longer than 50 amino acids
                start = lastStop + orf.find('M') * 3 if 'M' in orf else None # Update start codon position to next M, 
                stop = lastStop + (len(orf))*3 if 'M' in orf else None       # Update stop codon to next '*'
                if start is not None and stop is not None:                   # In the coding region is >30AA
                    orfs.append((start, stop))                                      # Append the start and stop positions for that ORF
            lastStop += len(orf)*3 + 3           # Update the position of the last stop codon
        orf_list.append(orfs)                    # Append the list of ORFs for the current frame
    return orf_list

def pickNLargestORF(orfs,N=5):
    Lengths = []
    Bounds = []
    for frame, orf in enumerate(orfs):
        for start, stop in orf:
            Lengths.append(stop-start)
            Bounds.append([start,stop])
    enumerated_list = list(enumerate(Lengths))
    sortedLengths = sorted(enumerated_list, key=lambda x: abs(x[1]), reverse=True)
    Bounds = [Bounds[index] for index, _ in sortedLengths[:N]]
    return Bounds

# Find all ORFs in each frame of the sequence
orf_positions = find_all_orfs(corona_virus.seq)

# Pick the largest ORF and translate it to amino acids
longestORFbounds = pickNLargestORF(orf_positions,10)
longestORFsequences = [corona_virus.seq[longestORFbounds[index][0]:].translate(to_stop=True) for index in range(10)]

# From last time, our protein of interest is the third largest ORF (i.e., sequence 2):
proteinOfInterest = longestORFsequences[2]

# 2) Creating and requesting a BLAST query.

The ```BLAST``` (Basic Local Alignment Search Tool) software is widely used in bioinformatics for comparing sequences, and its results are fundamental to the interpretation of a wide spectrum of biological data. In this case, we will make a BLAST using a module of Biopython, the ```Bio.Blast``` module.

In Python, BLAST process is asynchronous which means the script will need to wait and retrieve the result once it’s ready. 

The handle to the results is returned from the qblast() function and can then be parsed. The result would be in XML format which you would need to parse to get the actual BLAST_IDs and associated information.  The following section of the code can take different lengths of time depending on the numer of others who are using BLAST at the moment. Last time I ran this, it took 4 minutes, but it could take more or less time.

In [18]:
# Slice protein to only use the first 200 nucleotides
protein_sequence = proteinOfInterest[0:200]

# Set up the BLAST query for the NCBI database 
# This submits the query to the NCBI server for comparison to their database of sequences
# The 'protein-protein BLAST' (blastp) is used here
blast_result = NCBIWWW.qblast("blastp", "nr", protein_sequence)


## 2.A) Downloading the BLAST results

The BLAST was run online, and we need to retrieve the results before we can do anything with them.  

In [19]:
# Here, we download and save the BLAST results as an XML file.
blast_file_xml = "proteinBLASTresult.xml"
with open(blast_file_xml, "w") as out_handle:
    out_handle.write(blast_result.read())

Once we have downloaded them, we can convert them over into the FASTA format that we are more familiar with.

In [20]:
# In order to read the downloaded BLAST XML result file, it is useful to convert it to a FASTA file.
def convert_xml_2_fasta(blast_file_xml,blast_file_fasta):
    # Parse the BLAST XML file using NCBIXML         
    blast_records = NCBIXML.parse(open(blast_file_xml))
    with open(blast_file_fasta, "w") as handle:
        # Write the first sequence
        handle.write(f">Original\n{protein_sequence}\n")
        for blast_record in blast_records:
            for idx, alignment in enumerate(blast_record.alignments):
                for hsp in alignment.hsps:
                    # Extract the ID for each new sequence
                    ID = alignment.title.split('|')[:2]
                    if len(ID)>1:
                        ID = (ID[0]+'|'+ID[1])
                    else:
                        ID = ID[0]
                    # Write the seq ID and sequence to FASTA file
                    handle.write(f">{ID}\n{hsp.sbjct}\n")

blast_file_xml = "proteinBLASTresult.xml"
blast_file_fasta = "proteinBLASTresult.fa"
convert_xml_2_fasta(blast_file_xml,blast_file_fasta)                

### Exploring the contents of BLAST results

Now that we have BLAST results in XML and PASTA formats, let's look at the information we have collected.

First, lets look at what was contained in the BLAST XML that we downloaded from NCBI:

A ```BLAST XML file``` contains the results of a ```BLAST``` (Basic Local Alignment Search Tool) search. The file is structured in XML format and includes detailed information about the query sequence, the database searched, and the alignment results. Here is a description of the key elements typically found in a BLAST XML file:

### Key Elements:

1. **BlastOutput**: The root element containing the entire BLAST result.
   - **BlastOutput_program**: The name of the BLAST program used (e.g., blastn, blastp).
   - **BlastOutput_version**: The version of the BLAST program.
   - **BlastOutput_reference**: Reference information for the BLAST program.
   - **BlastOutput_db**: The database searched.
   - **BlastOutput_query-ID**: The identifier for the query sequence.
   - **BlastOutput_query-def**: The definition or description of the query sequence.
   - **BlastOutput_query-len**: The length of the query sequence.
   - **BlastOutput_param**: Parameters used for the BLAST search.

2. **BlastOutput_iterations**: Contains one or more iterations of the BLAST search.
   - **Iteration**: Represents a single iteration of the BLAST search.
     - **Iteration_iter-num**: The iteration number.
     - **Iteration_query-ID**: The identifier for the query sequence in this iteration.
     - **Iteration_query-def**: The definition or description of the query sequence.
     - **Iteration_query-len**: The length of the query sequence.
     - **Iteration_hits**: Contains one or more hits (matches) found in the database.
       - **Hit**: Represents a single hit.
         - **Hit_num**: The hit number.
         - **Hit_id**: The identifier for the hit sequence.
         - **Hit_def**: The definition or description of the hit sequence.
         - **Hit_accession**: The accession number of the hit sequence.
         - **Hit_len**: The length of the hit sequence.
         - **Hit_hsps**: Contains one or more HSPs (High-scoring Segment Pairs).
           - **Hsp**: Represents a single HSP.
             - **Hsp_num**: The HSP number.
             - **Hsp_bit-score**: The bit score of the HSP.
             - **Hsp_score**: The raw score of the HSP.
             - **Hsp_evalue**: The E-value of the HSP.
             - **Hsp_query-from**: The start position of the HSP in the query sequence.
             - **Hsp_query-to**: The end position of the HSP in the query sequence.
             - **Hsp_hit-from**: The start position of the HSP in the hit sequence.
             - **Hsp_hit-to**: The end position of the HSP in the hit sequence.
             - **Hsp_query-frame**: The reading frame of the query sequence.
             - **Hsp_hit-frame**: The reading frame of the hit sequence.
             - **Hsp_identity**: The number of identical matches in the HSP.
             - **Hsp_positive**: The number of positive matches in the HSP.
             - **Hsp_gaps**: The number of gaps in the HSP.
             - **Hsp_align-len**: The length of the alignment.
             - **Hsp_qseq**: The aligned part of the query sequence.
             - **Hsp_hseq**: The aligned part of the hit sequence.
             - **Hsp_midline**: The alignment midline showing matches, mismatches, and gaps.

### Example BLAST XML

To see an example, let's print the first 20 rows of the XML we just downloaded:


In [21]:
!head -n 20 "proteinBLASTresult.xml" 

<?xml version="1.0" encoding="US-ASCII"?>
<!DOCTYPE BlastOutput PUBLIC "-//NCBI//NCBI BlastOutput/EN" "http://www.ncbi.nlm.nih.gov/dtd/NCBI_BlastOutput.dtd">
<BlastOutput>
  <BlastOutput_program>blastp</BlastOutput_program>
  <BlastOutput_version>BLASTP 2.16.1+</BlastOutput_version>
  <BlastOutput_reference>Stephen F. Altschul, Thomas L. Madden, Alejandro A. Sch&amp;auml;ffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J. Lipman (1997), &quot;Gapped BLAST and PSI-BLAST: a new generation of protein database search programs&quot;, Nucleic Acids Res. 25:3389-3402.</BlastOutput_reference>
  <BlastOutput_db>nr</BlastOutput_db>
  <BlastOutput_query-ID>Query_1517829</BlastOutput_query-ID>
  <BlastOutput_query-def>unnamed protein product</BlastOutput_query-def>
  <BlastOutput_query-len>200</BlastOutput_query-len>
  <BlastOutput_param>
    <Parameters>
      <Parameters_matrix>BLOSUM62</Parameters_matrix>
      <Parameters_expect>10</Parameters_expect>
      <Parameters_gap-open>11</Par

### Using command line tools to quickly manipulate BLAST XML files.
Or, if we want to focus in on just a couple important aspects of the file, we can just grab those terms.  
Accession Numbers:
 -   ```!echo "proteinBLASTresult.xml" | xargs grep "<Hit_accession" | head -n 10```
Hit Scores
 -   ```!echo "proteinBLASTresult.xml" | xargs grep "Hsp_score" | head -n 10```
Hit Evalues (significance of alignment match)
 -   ```!echo "proteinBLASTresult.xml" | xargs grep "Hsp_evalue" | head -n 10```

In each of these:
* ```echo "proteinBLASTresult.xml"``` is printing the contents of the file into memory
* ```| xargs grep "BLAH"``` is piping the results into the ```grep``` function, which searches for lines that contain "BLAH"
* ```| head -n 10``` is piping the results into the ```head``` function, which returns the first n=10 rows.

Try it again but instead of piping into the head command, export (">") the result into a new text file:
 -   ```!echo "proteinBLASTresult.xml" | xargs grep "<Hit_accession" > dummy_file.txt``

Or, better yet, trim the file to get rid of all but the accession numbers before exporting:
-   ```!echo "proteinBLASTresult.xml" | xargs grep "<Hit_accession" | sed 's/.*>\(.*\)<.*/\1/' > "proteinBLASTresult_accessions.txt"```

In [22]:
!echo "proteinBLASTresult.xml" | xargs grep "<Hit_accession" | head -n 10

  <Hit_accession>URN79744</Hit_accession>
  <Hit_accession>URN80001</Hit_accession>
  <Hit_accession>URN79999</Hit_accession>
  <Hit_accession>URN79939</Hit_accession>
  <Hit_accession>UHB62636</Hit_accession>
  <Hit_accession>QKV38604</Hit_accession>
  <Hit_accession>7B62_A</Hit_accession>
  <Hit_accession>7L2C_A</Hit_accession>
  <Hit_accession>WVI77781</Hit_accession>
  <Hit_accession>QRX28344</Hit_accession>


In [23]:
!echo "proteinBLASTresult.xml" | xargs grep "<Hsp_score" | head -n 10

      <Hsp_score>1019</Hsp_score>
      <Hsp_score>1021</Hsp_score>
      <Hsp_score>1018</Hsp_score>
      <Hsp_score>1017</Hsp_score>
      <Hsp_score>1013</Hsp_score>
      <Hsp_score>1020</Hsp_score>
      <Hsp_score>1023</Hsp_score>
      <Hsp_score>1023</Hsp_score>
      <Hsp_score>1023</Hsp_score>
      <Hsp_score>1021</Hsp_score>


In [24]:
!echo "proteinBLASTresult.xml" | xargs grep "Hsp_evalue" | head -n 10

      <Hsp_evalue>2.14312e-138</Hsp_evalue>
      <Hsp_evalue>2.58465e-138</Hsp_evalue>
      <Hsp_evalue>2.6035e-138</Hsp_evalue>
      <Hsp_evalue>2.79284e-138</Hsp_evalue>
      <Hsp_evalue>9.82853e-138</Hsp_evalue>
      <Hsp_evalue>1.02309e-137</Hsp_evalue>
      <Hsp_evalue>1.26145e-137</Hsp_evalue>
      <Hsp_evalue>2.07528e-137</Hsp_evalue>
      <Hsp_evalue>3.34612e-137</Hsp_evalue>
      <Hsp_evalue>4.18214e-137</Hsp_evalue>


Now that you know how to grab different pieces of the information out of the XML, you can quickly create truncated data sets for whatever purpose you need.

For example, we can quickly write the FASTA file using a few simple commands:

In [32]:
# Make file of all accession numbers
# !echo "proteinBLASTresult.xml" | xargs grep "<Hit_accession" | sed 's/.*>\(.*\)<.*/\1/' > "proteinBLASTresult_accessions.txt"
# !echo "proteinBLASTresult.xml" | Select-String "<Hit_accession" | ForEach-Object { $_.Matches.Groups[1].Value } > "proteinBLASTresult_accessions.txt"
# Make file of all hit sequences
# !echo "proteinBLASTresult.xml" | xargs grep "<Hsp_hseq" | sed 's/.*>\(.*\)<.*/\1/' > "proteinBLASTresult_sequences.txt"
# !echo "proteinBLASTresult.xml" | Select-String "<Hsp_hseq" | ForEach-Object { $_.Matches.Groups[1].Value } > "proteinBLASTresult_sequences.txt"
# Combine the two files into a single FASTA file
with open("proteinBLASTresult_combined.fa","w") as combined:
    combined.write(f">Original\n{protein_sequence}\n")
    with open("proteinBLASTresult_accessions.txt") as accessions, open("proteinBLASTresult_sequences.txt") as sequences:
        for accession, sequence in zip(accessions, sequences):
            combined.write(f">{accession}")
            combined.write(sequence)

In [33]:
# Let's use our command line code 'head' to see the first several lines of the FASTA file that we 
# created using the first approach earlier:
!head "proteinBLASTresult.fa"

# Let's use our command line code 'head' to see the first several lines of the FASTA file that we
# created using the second approach with command line codes:
print("\n")
!head "proteinBLASTresult_combined.fa"

# After running the code above, we can see that the two FASTA files are the essentially same, but the 
# second approach is more efficient and easier to change for different files.


>Original
MFLLTTKRTMFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLRE
>gb|URN79744.1
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLRE
>gb|URN80001.1
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLRE
>gb|URN79999.1
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLRE
>gb|URN79939.1
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWM

In [34]:
# Now that we have a fasta file, we can run MUSCLE again to compute its alignment:
alignedFile = "blast_file_aligned.afa"

# Use a terminal command (bash) to run MUSCLE in to align the sequences.        
bashCommand = "muscle -align " + blast_file_fasta + " -output " + alignedFile
os.system(bashCommand)

0

In [35]:
# Make and save an alignment figure
mv = MsaViz(alignedFile, wrap_length=100, show_count=True)
mv.savefig("BLAST_Alignment.png")

# Load the image
img = mpimg.imread('BLAST_Alignment.png')

# Display the image
fig = plt.figure(figsize=(80, 80), dpi=200)
plt.imshow(img)
plt.axis('off')  # Hide axis
plt.show()

# 3) Running BLAST in a background thread
Often, we do not want to wait around until the blast call is complete.  For example, we may need to do this for a long list of sequences, or we might want to work on other tasks while we wait.  When this is the case, it can be helpful to run the code in the background on another thread.  As an example, the next cell shows an example where we run the BLAST on three different sequences all at the same time and in different threads.

For the three sequences, we will just use different lengths of the original.

In [ ]:
# Function to run a BLAST query and save the result to a file
def run_blast_query_and_save(query_sequence, result_file):
    result_handle = NCBIWWW.qblast("blastp", "nr", query_sequence)
    with open(result_file, "w") as out_handle:
        out_handle.write(result_handle.read())
    return

def run_a_set_of_blast_searches(pMOD_list):
    for idx, pMOD in enumerate(pMOD_list):
        query_sequence = pMOD[:200]  # Extract the first 200 amino acids from the protein sequence
        result_file = f"blast_result_{idx + 1}.xml"  # Generate a unique result file name
        blast_thread = threading.Thread(target=run_blast_query_and_save, args=(query_sequence, result_file))
        blast_thread.start()
        print("Started BLAST Job: " + result_file)
    return
        
# Assuming pMOD_list contains multiple protein sequences
pMOD_list = [protein_sequence, protein_sequence[25:175] , protein_sequence[50:150] ]  # Replace with your protein sequences

# Call function to run BLAST queries for each protein sequence in a separate thread
run_a_set_of_blast_searches(pMOD_list)